In [1]:
import pandas as pd
import numpy as np

## Load data

In [2]:
conf = pd.read_csv('/Users/Anna/Library/Mobile Documents/com~apple~CloudDocs/Kandidat/Speciale/Empirical Analysis - Mines and Conflict/Output/Grids/Conflicts in grid/Conf_Grid_20.csv')

In [5]:
conf = conf.rename(columns={'Grid_id':'Grid_ID'})

In [6]:
conf = conf.sort_values(['data_id', 'Grid_ID']).drop_duplicates('data_id') # remove duplicated (lowest Grid_ID - keeping the left or top grid)

In [7]:
len(conf) == 9693

True

In [8]:
mines_adm3 = pd.read_csv('/Users/Anna/Library/Mobile Documents/com~apple~CloudDocs/Kandidat/Speciale/Empirical Analysis - Mines and Conflict/Output/Grids/Mines in grid/Mines_Grid_20.csv')

In [11]:
mines_adm3 = mines_adm3.rename(columns={'Grid_id':'Grid_ID'})

In [12]:
mines_robust = pd.read_csv('mines_robust.csv')

In [19]:
mines = pd.merge(mines_robust, mines_adm3[['latitude', 'longitude', 'ADM3', 'Grid_ID']], how='left', on=['latitude', 'longitude'])

In [20]:
mines = mines[['Mineral', 'Grid_ID', 'ADM3']]

In [21]:
mines

,Mineral,Grid_ID,ADM3
0,Copper,4313.0,Kouga Local Municipality
1,Gold,4326.0,Bitou Local Municipality
2,Gold,4326.0,Bitou Local Municipality
3,Gold,4316.0,Bitou Local Municipality
4,Gold,4316.0,Bitou Local Municipality
...,...,...,...
1124,Copper,1057.0,Makhado Local Municipality
1125,Coal,982.0,Thulamela Local Municipality
1126,Coal,992.0,Mutale Local Municipality
1127,Coal,993.0,Mutale Local Municipality


In [22]:
mines.Mineral = mines.Mineral.apply(lambda x: str(x)+str('_min'))

In [23]:
prices = pd.read_csv('monthly_prices_log.csv')
prices = prices.drop(columns='Unnamed: 0')

In [25]:
grids = pd.read_csv('/Users/Anna/Library/Mobile Documents/com~apple~CloudDocs/Kandidat/Speciale/Empirical Analysis - Mines and Conflict/Output/Grids/grid_w_SOU_att/Grid_20_ADM3.csv')

In [27]:
grids = grids.rename(columns={'Grid_id':'Grid_ID'})

In [28]:
grids = grids [['Grid_ID', 'ADM3']]

## Montly conflict data

In [29]:
conf['count'] = 1

In [30]:
conf_types = pd.pivot_table(conf, values='count', index=['Grid_ID', 'event_date'], columns=['event_type'], aggfunc=np.sum, fill_value=0).reset_index()

In [31]:
65+9+4771+4228+38+582

9693

In [32]:
pd.to_datetime(conf_types.event_date)

0      2015-10-30
1      2012-08-25
2      2016-01-06
3      2016-06-30
4      2016-05-31
          ...    
8883   2019-11-21
8884   2014-08-31
8885   2018-11-13
8886   2018-11-14
8887   2017-08-26
Name: event_date, Length: 8888, dtype: datetime64[ns]

In [33]:
conf_types['year_month'] = pd.to_datetime(conf_types['event_date']).dt.to_period('M')

In [34]:
conf_types.head(3)

event_type,Grid_ID,event_date,Battles,Explosions/Remote violence,Protests,Riots,Strategic developments,Violence against civilians,year_month
0,8,30/10/2015,0,0,0,1,0,0,2015-10
1,15,25/08/2012,0,0,0,1,0,0,2012-08
2,22,01/06/2016,0,0,0,1,0,0,2016-01


In [35]:
conf_types = conf_types.groupby(["Grid_ID","year_month"]).sum().reset_index()

In [36]:
conf_types.sum()

event_type
Grid_ID                       14316767
Battles                             65
Explosions/Remote violence           9
Protests                          4771
Riots                             4228
Strategic developments              38
Violence against civilians         582
dtype: int64

In [37]:
65+9+4771+4228+38+582 # check sum

9693

### Type of mines within admin boundaries

In [38]:
mines_admin = mines.groupby("Grid_ID")['Mineral'].value_counts().unstack(fill_value=0).reset_index()

### All dates for each admin boundary

In [39]:
grids = grids.dropna(subset=['ADM3']) # only keep grids that are interacted with AM#, and thus inside the country boarde

In [40]:
grids

,Grid_ID,ADM3
0,1,Cobham State Forest
1,2,Cobham State Forest
2,3,Cobham State Forest
3,4,Cobham State Forest
4,5,Cobham State Forest
...,...,...
4663,4664,Kgalagadi Transfrontier P
4664,4665,Kgalagadi Transfrontier P
4665,4666,Kgalagadi Transfrontier P
4666,4667,Kgalagadi Transfrontier P


In [41]:
print(len(grids.Grid_ID.unique())) # number of unique local municipalities
unique_grids = grids.Grid_ID.unique()

4668


In [42]:
prices_adm = prices.copy()

In [43]:
s= pd.Series(unique_grids) 
s_n = len(s)
prices_adm_n = len(prices_adm)
prices_adm = pd.DataFrame(np.repeat(prices_adm.values,s_n, axis = 0), columns = prices_adm.columns, index = np.tile(s,prices_adm_n)).rename_axis('Grid_ID').reset_index()

In [44]:
len(prices_adm) == 96 * len(unique_grids) # Check if correct number of rows has been made (96 number of periods)

True

In [45]:
len(prices_adm.Grid_ID.unique()) == len(unique_grids) # Check if all admins have been included

True

In [46]:
prices_adm

,Grid_ID,year_month,Aluminium_price,Copper_price,Gold_price,Lead_price,Nickel_price,Platinum_price,Uranium_price,Zinc_price,Iron_price,Coal_price,Titanium_price,Diamond_price
0,1,2012-01,3.33748,3.90544,3.21921,3.32312,4.29961,3.17838,1.75821,3.29938,2.14693,2.02637,1.06209,2.36847
1,2,2012-01,3.33748,3.90544,3.21921,3.32312,4.29961,3.17838,1.75821,3.29938,2.14693,2.02637,1.06209,2.36847
2,3,2012-01,3.33748,3.90544,3.21921,3.32312,4.29961,3.17838,1.75821,3.29938,2.14693,2.02637,1.06209,2.36847
3,4,2012-01,3.33748,3.90544,3.21921,3.32312,4.29961,3.17838,1.75821,3.29938,2.14693,2.02637,1.06209,2.36847
4,5,2012-01,3.33748,3.90544,3.21921,3.32312,4.29961,3.17838,1.75821,3.29938,2.14693,2.02637,1.06209,2.36847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448123,4664,2019-12,3.25089,3.78263,3.16981,3.27861,4.13989,2.96597,1.4067,3.3566,1.96685,1.88098,1.0328,2.32222
448124,4665,2019-12,3.25089,3.78263,3.16981,3.27861,4.13989,2.96597,1.4067,3.3566,1.96685,1.88098,1.0328,2.32222
448125,4666,2019-12,3.25089,3.78263,3.16981,3.27861,4.13989,2.96597,1.4067,3.3566,1.96685,1.88098,1.0328,2.32222
448126,4667,2019-12,3.25089,3.78263,3.16981,3.27861,4.13989,2.96597,1.4067,3.3566,1.96685,1.88098,1.0328,2.32222


In [47]:
pd.to_datetime(prices_adm.year_month, format='%Y-%m')
prices_adm['year_month'] = pd.to_datetime(prices_adm['year_month']).dt.to_period('M')

In [48]:
#prices_adm = pd.merge(prices_adm, grids, how='left', on='Grid_ID')
#prices_adm

### Merge admin mines and mine prices together

In [49]:
mines_prices = pd.merge(prices_adm, mines_admin,how='left', on ='Grid_ID')

In [50]:
mines_prices_conf = pd.merge(mines_prices, conf_types, how='left', on=['year_month', 'Grid_ID'])

In [51]:
#mines_prices_conf.sum()

In [52]:
mines_prices_conf = pd.merge(mines_prices_conf, grids, on='Grid_ID', how='left')

In [53]:
mines_prices_conf.sort_values(by=['Grid_ID', 'year_month'])

,Grid_ID,year_month,Aluminium_price,Copper_price,Gold_price,Lead_price,Nickel_price,Platinum_price,Uranium_price,Zinc_price,...,Titanium_min,Uranium_min,Zink_min,Battles,Explosions/Remote violence,Protests,Riots,Strategic developments,Violence against civilians,ADM3
0,1,2012-01,3.33748,3.90544,3.21921,3.32312,4.29961,3.17838,1.75821,3.29938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cobham State Forest
4668,1,2012-02,3.3514,3.92545,3.24094,3.32682,4.3096,3.22063,1.75603,3.31359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cobham State Forest
9336,1,2012-03,3.34734,3.92722,3.22429,3.31335,4.27106,3.21985,1.7466,3.3088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cobham State Forest
14004,1,2012-04,3.31953,3.91696,3.21759,3.31679,4.25268,3.20076,1.7452,3.30161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cobham State Forest
18672,1,2012-05,3.30997,3.89962,3.20075,3.30072,4.22944,3.16687,1.74851,3.28485,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cobham State Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429455,4668,2019-08,3.24802,3.75659,3.17456,3.31064,4.19688,2.93536,1.40281,3.3568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kgalagadi Transfrontier P
434123,4668,2019-09,3.25027,3.76037,3.178,3.31636,4.24734,2.97411,1.40492,3.36765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kgalagadi Transfrontier P
438791,4668,2019-10,3.23891,3.75941,3.17541,3.33945,4.23293,2.95405,1.39304,3.3882,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kgalagadi Transfrontier P
443459,4668,2019-11,3.24742,3.76799,3.16935,3.30776,4.18213,2.95592,1.40106,3.38577,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kgalagadi Transfrontier P


In [54]:
mines_prices_conf.columns

Index(['Grid_ID', 'year_month', 'Aluminium_price', 'Copper_price',
       'Gold_price', 'Lead_price', 'Nickel_price', 'Platinum_price',
       'Uranium_price', 'Zinc_price', 'Iron_price', 'Coal_price',
       'Titanium_price', 'Diamond_price', 'Aluminium_min', 'Coal_min',
       'Copper_min', 'Diamond (Alluvial)_min', 'Diamond (Kimberline)_min',
       'Gold_min', 'Iron_min', 'Lead_min', 'Nickel_min', 'Platinum_min',
       'Titanium_min', 'Uranium_min', 'Zink_min', 'Battles',
       'Explosions/Remote violence', 'Protests', 'Riots',
       'Strategic developments', 'Violence against civilians', 'ADM3'],
      dtype='object')

In [55]:
mines_prices_conf['year_month']=mines_prices_conf['year_month'].astype(str)
mines_prices_conf.iloc[:,2:-1]=mines_prices_conf.iloc[:,2:-1].astype(float)
mines_prices_conf['ADM3']=mines_prices_conf['ADM3'].astype(str)

In [56]:
mines_prices_conf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448128 entries, 0 to 448127
Data columns (total 34 columns):
Grid_ID                       448128 non-null int64
year_month                    448128 non-null object
Aluminium_price               448128 non-null float64
Copper_price                  448128 non-null float64
Gold_price                    448128 non-null float64
Lead_price                    448128 non-null float64
Nickel_price                  448128 non-null float64
Platinum_price                448128 non-null float64
Uranium_price                 448128 non-null float64
Zinc_price                    448128 non-null float64
Iron_price                    448128 non-null float64
Coal_price                    448128 non-null float64
Titanium_price                448128 non-null float64
Diamond_price                 448128 non-null float64
Aluminium_min                 48384 non-null float64
Coal_min                      48384 non-null float64
Copper_min                    

In [82]:
mines_prices_conf['Diamond_min'] = mines_prices_conf['Diamond (Alluvial)_min'] + mines_prices_conf['Diamond (Kimberline)_min']

mines_prices_conf = mines_prices_conf.drop(columns=['Diamond (Kimberline)_min', 'Diamond (Alluvial)_min'])

In [57]:
mines_prices_conf.to_csv('dataset_mines_prices_conf_GRID20_ROBUST.csv')